In [ ]:
import pandas as pd
import hvplot.pandas

In [ ]:
QUOTA = pd.DataFrame(data={"NodeHours":[8000]*12}, index=pd.date_range("2025-02-14",freq="MS",periods=12))
QUOTA["CumNodeHours"] = QUOTA["NodeHours"].cumsum()
PLAN = pd.DataFrame(data={"NodeHours":[3000]*3+[10000]*9}, index=pd.date_range("2025-02-14",freq="MS",periods=12))
PLAN["CumNodeHours"] = PLAN["NodeHours"].cumsum()

In [ ]:
brey = pd.read_csv("sacct_energy_nodehours_brey.txt", sep="\t")
panos = pd.read_csv("sacct_energy_nodehours_panos.txt", sep="\t")
tomsail = pd.read_csv("sacct_energy_nodehours_tomsail.txt", sep="\t")

def add_info(df:pd.DataFrame, name):
    df["Submit"] = pd.to_datetime(df["Submit"], errors="coerce")
    df.index = df["Submit"]
    df = df[df["NodeHours"]>0]
    df =df.rename(columns={
        "NodeHours":f"{name} usage",
        })
    return df
brey = add_info(brey,"u101116")
tomsail = add_info(tomsail,"u101170")
panos = add_info(panos,"u101348")

In [ ]:
usage = pd.concat([brey, tomsail, panos],axis=0).fillna(0)
usage = usage.sort_index()

bar = usage[["u101170 usage","u101116 usage","u101348 usage"]].resample("MS").sum().hvplot.bar(stacked=True)
(QUOTA["NodeHours"].hvplot.bar(c="grey",alpha=0.5, label="quota") * bar).opts(width=900, title ="Monthly usage, compared to quota allocated")
(PLAN["NodeHours"].hvplot.bar(c="grey",alpha=0.5, label="flexible quota") * bar).opts(width=900, title ="Monthly usage, compared to requested quota in proposal")

In [ ]:
usage[["u101116 cumulated","u101348 cumulated", "u101170 cumulated"]] = usage[["u101116 usage","u101348 usage", "u101170 usage"]].cumsum()

In [ ]:
usage = usage[~usage.index.duplicated(keep="first")]

In [ ]:
QUOTA = QUOTA.rename(columns={"CumNodeHours":"Allocated Quota"})
PLAN = PLAN.rename(columns={"CumNodeHours":"Requested Quota"})
(usage[["u101116 cumulated","u101170 cumulated","u101348 cumulated", ]].hvplot.area() * 
 QUOTA["Allocated Quota"].hvplot.line(c='k', line_dash = "dashed") * 
 PLAN["Requested Quota"].hvplot.line(c='grey', line_dash = "dashed")).opts(
    xlim=(None, pd.Timestamp.now()),
    ylim=(0,50000),
    width=1000,
    height=600,
    )

In [ ]:
energy = (brey.ConsumedEnergyRaw.sum() + tomsail.ConsumedEnergyRaw.sum() + panos.ConsumedEnergyRaw.sum()) / 3.6e6
f"energy consumed: {energy:.2f} kWh"
co2 = energy * 409.8438/1000
f"corresponding to {co2:.2f} CO2 kg Emissions"